In [ ]:
from finbourne_sdk_utils.jupyter_tools import toggle_code

"""Luminesce Syntax

Attributes
----------

"""

toggle_code("Toggle Docstring")


*Setting up the environment for the tutorial*

This section of the notebook creates a portfolio, some equities, and a file in drive to run luminesce queries against.

In [ ]:
from lusid.extensions import (
    SyncApiClientFactory,
    ArgsConfigurationLoader,
    EnvironmentVariablesConfigurationLoader,
    SecretsFileConfigurationLoader
)
from lusidjam import RefreshingToken
import os
import lusid as lu
import lusid.models as lm
import pandas as pd
import lusid_drive
import lusid_drive.rest
import lusid.rest
from lusid_drive.rest import ApiException
from pprint import pprint
import json

# Authenticate to SDK
# Run the Notebook in Jupyterhub for your LUSID domain and authenticate automatically
secrets_path = os.getenv("FBN_SECRETS_PATH")
# Run the Notebook locally using a secrets file (see https://support.lusid.com/docs/how-do-i-use-an-api-access-token-with-the-lusid-sdk)
if secrets_path is None:
    secrets_path = os.path.join(os.path.dirname(os.getcwd()), "secrets.json")

# Initiate an API Factory which is the client side object for interacting with LUSID APIs
config_loaders=[
    ArgsConfigurationLoader(access_token = RefreshingToken(), app_name = "LusidJupyterNotebook"),
    EnvironmentVariablesConfigurationLoader(),
    SecretsFileConfigurationLoader(secrets_path)]
api_factory = lusid.extensions.SyncApiClientFactory(config_loaders=config_loaders)
 

# Confirm success by printing SDK version
api_status = pd.DataFrame(api_factory.build(lu.ApplicationMetadataApi).get_lusid_versions().to_dict())
display(api_status)

In [31]:
# here's some sample equities we've prepared earlier
uk_equity_instruments = pd.read_csv('_data/uk_equity_instruments.csv')
us_equity_instruments = pd.read_csv('_data/us_equity_instruments.csv')

In [32]:
# don't show exception if error is due to upsert
def exception_guard(e, code):
    return e.status and e.status != '400 Bad Request' and e.body and json.loads(e.body)["code"] == code

scope = 'FBNUniversityModule-T2-2' # str | The scope of the portfolio.

In [33]:
# create a request to upsert a bunch of instruments.
def generate_upsert_instrument_request(api_instance, name, ccy, figi, isin, other_identifier_name, other_identifier):
    request_id = name + isin
    request ={"name":name,
              "identifiers":{ 
                  "Figi":{"value":figi},
                  "Isin":{"value":isin},
                  other_identifier_name:{"value":other_identifier}
                },
              "definition":lm.Equity(
                              instrument_type = "Equity",
                              dom_ccy = ccy,
                              identifiers = {}
                          )
    }
    return request_id, request

In [34]:
# Create an instance of the API class
api_instance = api_factory.build(lu.InstrumentsApi)

# We're going to load some american and UK equities into LUSID

other_identifier_name = 'Sedol'
ccy = 'GBP'
upsert_body_tuple = (generate_upsert_instrument_request(api_instance, name, ccy, figi, isin, other_identifier_name, other_identifier)
               for 
               index, ticker, name, _, isin, other_identifier, figi
               in uk_equity_instruments.itertuples())
uk_equity_body_dict = {request_id:request_body for request_id,request_body in upsert_body_tuple}
other_identifier_name = 'Cusip'
ccy = 'USD'
upsert_body_tuple = (generate_upsert_instrument_request(api_instance, name, ccy, figi, isin, other_identifier_name, other_identifier)
               for 
               index, ticker, name, _, isin, other_identifier, figi
               in us_equity_instruments.itertuples())
us_equity_body_dict = {request_id:request_body for request_id,request_body in upsert_body_tuple}

request_body_dict = us_equity_body_dict | uk_equity_body_dict

try:
    # UpsertInstruments: Upsert instruments
    api_response = api_instance.upsert_instruments(request_body_dict, scope=scope)
#     pprint(api_response)
except lusid.rest.ApiException as e:
    print("Exception when calling InstrumentsApi->upsert_instruments: %s\n" % e)


In [ ]:

from lusid_drive.extensions import (
    SyncApiClientFactory,
    ArgsConfigurationLoader,
    EnvironmentVariablesConfigurationLoader,
    SecretsFileConfigurationLoader
)
from lusid_drive.models import *

# Authenticate to SDK
# Run the Notebook in Jupyterhub for your LUSID domain and authenticate automatically
secrets_path = os.getenv("FBN_SECRETS_PATH")
api_url_drive = os.getenv("FBN_DRIVE_API_URL")
# Run the Notebook locally using a secrets file (see https://support.lusid.com/docs/how-do-i-use-an-api-access-token-with-the-lusid-sdk)
if secrets_path is None:
    secrets_path = os.path.join(os.path.dirname(os.getcwd()), "secrets.json")

# Initiate an API Factory which is the client side object for interacting with LUSID APIs
config_loaders_drive=[
    ArgsConfigurationLoader(access_token = RefreshingToken(), app_name = "LusidJupyterNotebook", api_url=api_url_drive),
    EnvironmentVariablesConfigurationLoader(),
    SecretsFileConfigurationLoader(secrets_path)]
api_factory_drive = lusid_drive.extensions.SyncApiClientFactory(config_loaders=config_loaders_drive)
 
folders_api = api_factory_drive.build(lusid_drive.api.FoldersApi)

# let's chuck a file containing a randomly selected subset of 25 instruments with their sectors into drive for some examples:
subset_file = open('_data/instrument_subset.csv').read()
create_folder = {"path":"/","name":"FBN-University"} # CreateFolder | A CreateFolder object that defines the name and path of the new folder

try:
    # [EARLY ACCESS] CreateFolder: Create a new folder in LUSID Drive
    api_response = folders_api.create_folder(create_folder)
    pprint(api_response)
except ApiException as e:
    if not exception_guard(e, 664):
        print("Exception when calling FilesApi->create_file: %s\n" % e)
    
files_api = api_factory_drive.build(lusid_drive.api.FilesApi)
    
x_lusid_drive_filename = 'instrument_subset.csv' # str | File name.
x_lusid_drive_path = '/FBN-University/' # str | File path.
content_length = len(subset_file.encode('UTF-8'))
body = subset_file # str | 

try:
    # [EARLY ACCESS] CreateFile: Uploads a file to Lusid Drive. If using an SDK, consider using the UploadAsStreamAsync function for larger files instead.
    api_response = files_api.create_file(x_lusid_drive_filename, x_lusid_drive_path, content_length, body)
except lusid_drive.rest.ApiException as e:
    if not exception_guard(e, 671):
        print("Exception when calling FilesApi->create_file: %s\n" % e)
        

# Luminesce Syntax

In this part of the course we will provide some additional technical information about luminesce, and demonstrate how luminesce queries can be used to extract information from your data.

## What is Structured Query Language (SQL)


SQL - pronounced "sequel" was first developed at IBM in the early 1970s, as a way to query and manipulate structured relational databases. SQL is a set-based, declarative programming language. Many implementations of SQL add extensions with procedural programming functionality, such as control-of-flow contructs.

## Luminesce Queries - SQL with extensions


SQLite is the most widely deployed database engine, used by several of the top web browsers, operating systems and application software.

Luminesce supports most of the query syntax for the SQLite dialect of SQL, but not the data manipulation or data definition syntax. Data can still be manipulated using some extensions to the language that Luminesce adds. 

Luminesce adds extensions to the SQLite implementation which can:
- exclude columns from queries
- automatically select important columns in queries
- parameterise queries
- write data to some Luminesce providers
- wait on results of some dependent operation, or for a time period
- run queries in parallel on an iterated range of parameters.



## Luminesce providers


A provider is a component that enables you to write a Luminesce SQL query for a data source in situ, without first having to extract, transform or load data from that source.

We supply providers for numerous data sources, including for the investment data stored in LUSID itself, for files stored in Drive and for some external data providers. You can also create your own provider for a proprietary data source.

There are two types of provider: a data provider and a direct provider.

A data provider is designed to query a data source whose shape is known. It therefore returns a table of results with a fixed number of fields (columns).

A direct provider is designed to query a data source whose shape is not known, and thus cannot return a table of results with a fixed number of fields (columns).

We'll use both data and direct providers in this tutorial.

## Use case: Using Luminesce to query and join data from disparate sources, and to distribute the resulting information.


I'd like to read a file from Drive and use the data from this file to filter transaction results. I'd like to join these filtered results with another provider and distribute the results. This will be packaged up into a simple to use View which runs on a schedule.

## An intro to SQLite queries

In [ ]:
# Only required if running locally:
!pip3 install -U lumipy

In [37]:
## Only required if running locally:
import os
from IPython.core.magic import (register_line_cell_magic)
from lumipy.client import Client
from lusidjam import RefreshingToken

token = RefreshingToken()
lumi_url = os.getenv("FBN_LUMI_API_URL")

@register_line_cell_magic
def luminesce(line, cell=None):
    query = cell if cell is not None else line

    try:
        lm_client = Client(token=token, api_url=lumi_url)
    except TypeError:
        # Attempt to use V2 SDK syntax if V1 syntax fails.
        # This gives V2 SDK support for luminesce magic
        lm_client = Client(access_token=token, api_url=lumi_url)

    df = lm_client.query_and_fetch(query)
            
    return df

# In an interactive session, we need to delete to avoid name conflicts for automagic to work on line magics.
del luminesce

### A query using a simple SELECT statement


Its simple to get data from a provider using a select statement.

Use the following statement to get all data from a provider:

`SELECT * FROM SOME.PROVIDER`

Or retrieve data from a subset of columns by specifying them:

`SELECT A_COLUMN, ANOTHER_COLUMN FROM SOME.PROVIDER`

Here we'll use a SELECT statement to query for all of our porfolios.

In [ ]:
%%luminesce
SELECT * 
FROM Lusid.Instrument.Equity
LIMIT 100

In this example, we've selected all columns and rows from the Lusid.Instrument.Equity provider. We can specify column names if we are only interested in some parts of the data.

In [ ]:
%%luminesce
SELECT DisplayName, Figi 
FROM Lusid.Instrument.Equity
LIMIT 100

Here I really just want to see the name of the Equity and the Figi, I've selected only those columns from the provider.

### Limiting the number of return results using LIMIT


These simple SELECT statements are useful for getting an idea of the shape and quality of the data returned by a provider, but they tend to return a lot of data. Lets use LIMIT to reduce the amount of data requested:

In [ ]:
%%luminesce
SELECT DisplayName, Figi 
FROM Lusid.Instrument.Equity 
LIMIT 5

This example returns the first 5 rows of data from the Lusid.Instrument.Equity provider.

### Filter queries WHERE ...


We can now filter these results down - lets get all of the Equities that are listed in GBP


In [ ]:
%%luminesce
SELECT * 
FROM Lusid.Instrument.Equity 
WHERE DomCcy = 'GBP'
LIMIT 100

We've filtered our query for equities who have a domestic currency of Pound Sterling.

### JOINing disparate datasources with join


Now, I've got a file in drive with sector information for some of these instruments. I'm going to join data from the file using our Drive provider with the saved instrument data so I can see the sectors for instruments where we know this information.

First lets get our instrument data.

Then we'll grab the file contents from drive.

We'll take a look at cross-joins, which SQLite uses to build left joins and inner joins. Cross-joins return the cartesian product of both tables - each row on the left table is concatenated with every row on the right table, to form a massive table with every combination of both tables' rows:


In [ ]:
%%luminesce

/*
    We'll cover using variables later in the tutorial
*/

@x = use Drive.Csv
--file=/finbourne university/instrument_subset.csv
enduse;

SELECT DisplayName, DomCcy, File.Sector 
FROM Lusid.Instrument.Equity as Equities 
CROSS JOIN @x as File
LIMIT 100

We've returned a table with every combination of Lusid Equities with every row in our file.

In a left join, all rows from the left table are returned, along with rows in the right table that match the supplied key constraint. If there is a row in the left table with no matching row on the right, the columns that come from the right table in the resulting row are set to null.

In [ ]:
%%luminesce

/*
    We'll cover using variables later in the tutorial
*/

@x = use Drive.Csv
--file=/finbourne university/instrument_subset.csv
enduse;

SELECT DisplayName, DomCcy, File.Sector, Equities.Figi as Figi 
FROM @x as File
LEFT OUTER JOIN Lusid.Instrument.Equity as Equities
on Equities.Figi = File.Figi
LIMIT 100

We see this here as there are some companies with no matching sector in our file - these are returned as null.

We are going to ignore instruments without a matching sector for now, so we'll use an inner join.
Inner joins only return rows where the joining key has a match on both sides of the table:

In [ ]:
%%luminesce

/*
    We'll cover using variables later in the tutorial
*/

@x = use Drive.Csv
--file=/finbourne university/instrument_subset.csv
enduse;

SELECT DisplayName, DomCcy, File.Sector, Equities.Figi as Figi 
FROM @x as File 
INNER JOIN Lusid.Instrument.Equity as Equities  on File.Figi = Equities.Figi 
LIMIT 100


Now we have a joined table where only Equities with a sector on file are returned, along with their sector.

### GROUPing results BY some condition in query


We can also group our results by a column - we'll perform an aggregation and group-by to count the number of instrument in each sector.

There are many different aggregation functions in SQLite, such as sum, avg and count.

Sum will return the sum of the specified field in each group.
Average returns the mean of the specified field in each group.
Count returns the number of results that are non-null for the field in each group. We can also specify that count returns the count of distinct values [Count(Distinct <field>)], or all rows [count(*)], in each group.

In [ ]:
%%luminesce

/*
    We'll cover using variables later in the tutorial
*/

@x = use Drive.Csv
--file=/finbourne university/instrument_subset.csv
enduse;

SELECT count(File.Sector), File.Sector
FROM @x as File
LEFT OUTER JOIN Lusid.Instrument.Equity as Equities
on Equities.Figi = File.Figi
group by File.Sector
LIMIT 100

We've now grouped our results by sector, and for each sector, we''ve returned how many Equities belong to the sector.

## Introducing some Luminesce extensions


### Using ^ to SELECT the most important columns from a provider


Let's use the carat symbol to grab then more important columns in the Equity provider. The list of important columns is decided by the writer of the Provider.

In [ ]:
%%luminesce
SELECT ^ 
FROM Lusid.Instrument.Equity
LIMIT 100

In the Lusid Equities provider: LusidInstrumentId, Scope, DisplayName, InferredAssetClass, InferredDomCcy and IsActive are the fields marked by the provider as being most important.

### Excluding columns from the results of a query using the except keyword.


Using some providers, we can also exclude columns from the result of a query by using the except keyword:

In [ ]:
%%luminesce
SELECT ^ (except Type, IsActive) 
FROM Lusid.Instrument.Equity
LIMIT 100

Here we select the most important columns again, but exclude the Type and IsActive columns

### Using parameters to filter queries


Many providers will have parameters that can also be used to filter a Luminesce query as part of a standard WHERE clause.

We can see these parameters by querying the Sys.Field Provider.

In [ ]:
%%luminesce
SELECT FieldName, DataType, ParamDefaultValue, Description 
FROM Sys.Field 
WHERE TableName = 'Lusid.Instrument.Equity' 
AND FieldType = 'Parameter'
LIMIT 100

We can see there are 3 fields we can filter our query on: AsAt, EffectiveAt and UseLusidFilter

In [ ]:
%%luminesce

SELECT * 
FROM Lusid.Instrument.Equity
WHERE UseLusidFilter = False
LIMIT 100

Here we've filtered an Equity query on the UseLusidFilter. Notice there is no returned Field called UseLusidFilter - its not a column in our data.

## Using Luminesce variables


You can use variables to create and populate arbitrary tables of data that can then be used as part of a Luminesce query for either a data provider or a direct provider.

Declare a variable that represents a table of data using:
`@variable`

Or to represent a single value:
`@@variable`

Let's see variables in action. 

In [ ]:
%%luminesce
@instruments_in_GBP = SELECT * 
                      FROM Lusid.Instrument.Equity 
                      WHERE DomCcy = 'GBP'
                      LIMIT 100;
SELECT Isin 
FROM @instruments_in_GBP;

Here_instruments_in GBP is a table variable, set from the results of the query on equities.

In [ ]:
%%luminesce
@sectors = use Drive.Csv
--file=/finbourne university/instrument_subset.csv
enduse;

@@selected_sector = SELECT 'Life Insurance';
SELECT * 
FROM @sectors 
WHERE Sector = @@selected_sector
LIMIT 100

I've set selected_sector as a single-value variable "Lise Insurance", and filtered my file by sector.

## Introducing more advanced Luminesce extensions


You can use the CROSS APPLY or OUTER APPLY statement to iterate over a set of parameters. This is useful to execute a Luminesce query in parallel for each of the parameter inputs.

### Cross-apply

Cross-apply is similar to INNER JOIN, in that only records that have matching values on both sides are returned. 
We'll run some equity queries in parallel using Tools.split to create a set of currencies to filter by- the table will contain the currency joined to all of the returned equity tables:

In [ ]:
%%luminesce

select VALUE as val
from Tools.Split 
where Original = 'GBP USD CHF' 
AND Delimiters = ' ';


First, we use Tools.Split to turn a space separated string of currencies into into a table where each currency is an entry in the table.

In [ ]:
%%luminesce

@currencies = select VALUE as val
              from Tools.Split 
              where Original = 'GBP USD CHF' 
              AND Delimiters = ' ';

SELECT c.val as selectedCurrency, results.*
FROM @currencies c
CROSS APPLY (
       SELECT equity.^ 
       FROM Lusid.Instrument.Equity equity
       WHERE equity.DomCcy = c.val
       LIMIT 100       
) results


We then save the result of that query in a variable @currency, which is passed as a criterion in the CROSS APPLY subquery,  which runs in parallel for each row in @currency.

This example works similar to an inner join, we've run a query in parallel for some currency values, returning a similar table to an inner join. Remember, inner join returns only rows which fulfil the join criteria.

### Outer-apply


Outer apply is similar to LEFT OUTER JOIN, in that all records on the left side are returned, even when there are no matching records on the right side.

In [ ]:
%%luminesce

@currencies = select VALUE as val
              from Tools.Split 
              where Original = 'GBP USD CHF' 
              AND Delimiters = ' ';
              
SELECT c.val, results.*
FROM @currencies c
OUTER APPLY (
       SELECT equity.^ 
       FROM Lusid.Instrument.Equity as equity
       WHERE equity.DomCcy = c.val
       LIMIT 100
) results


We've run a query in parallel for some currency values, returning a similar table to a left join with our selected currencies on the left side of the join. Here, the values from Lusid.Instrument.Equity on the CHF row are null.